## =====Parameter Definition=====

In [1]:
# https://github.com/zake7749/PTT-Crawler



import os

# ----- ptt_board to crawl
ptt_board = 'Salary'
data_root = os.path.abspath(os.getcwd())+ '/data/'
raw_data_path = data_root+'ptt_'+ptt_board+'/raw/'

# ----- data with basic processing
# processed_data_path = data_root+'processed_sex/'
processed_data_path = data_root+'ptt_'+ptt_board+'/processed/'

# structurized csv files
csv_path = processed_data_path + ptt_board + '.csv'
# # Data Cleaning and Tokenizing
# tok_csv_path = processed_data_path + ptt_board + '_tok' + '.csv'
w2v_csv_path = processed_data_path + ptt_board + '_w2v' + '.csv'

model_path = './models/' + ptt_board + '/'
model_name = ptt_board + 'w2v_model_all.md'

if not os.path.isdir(model_path):
    os.makedirs(model_path)
    print("made folder:", model_path)

In [2]:
# print(os.path.abspath(os.getcwd()))

## =====Run Crawler & ETL===== 

 ## Data Collection

In [ ]:
# # Crawl data from ptt according to given board. 
# # you can also edit Crawler.py and run it in the shell 

# from Crawler import PttCrawler

# crawler = PttCrawler()
# crawler.crawl(board=ptt_board, start=2, end=5)

## Data Preprocessing

In [ ]:
# ##--- process_raw_data

# from filter import ArticleFilter

# Filter = ArticleFilter(data_root, processed_data_path)

# Filter.process_raw_data(raw_data_path, is_dir=True,to_one_file=True,one_file_name="CorpusPatch2.json")
# Filter.merge_coprus()

In [ ]:
##--- load processed data for further cleaning

# from filter import ArticleFilter

# Filter = ArticleFilter(data_root, processed_data_path)

# Filter.load_processed_corpus(processed_data_path)
# # Filter.print_titles()
# # Filter.print_response()
# # Filter.print_user_info() TODO?


In [ ]:
# print(len(Filter.order_response))
# print(Filter.order_titles[0])
# print(Filter.order_content[0])
# print(Filter.order_response[0])
# print(Filter.order_votes[0])
# print(Filter.order_users[0])
# print(Filter.order_users[0])

In [ ]:
# import pandas as pd

# pd_corpus = pd.DataFrame(data={'title': Filter.order_titles, 'content': Filter.order_content, 'response':Filter.order_response,
#                               'vote': Filter.order_votes, 'user': Filter.order_users, 'date': Filter.order_date,})

# pd_corpus.head()

In [ ]:
# # pd_corpus.title = pd_corpus.title.str.encode('utf-8')
# # pd_corpus.content = pd_corpus.content.str.encode('utf-8')
# # pd_corpus.response = pd_corpus.response.str.encode('utf-8')

# pd_corpus['Re'] = False
# pd_corpus['Fw'] = False

# pd_corpus.loc[pd_corpus.title.str.contains('Re'), 'Re'] = True
# pd_corpus.loc[pd_corpus.title.str.contains('Fw'), 'Fw'] = True

# pd_corpus.head()

In [ ]:
# pd_corpus.to_csv(csv_path)

## =====TF-IDF=====

In [4]:
!pip install snownlp

     |████████████████████████████████| 37.6 MB 8.8 MB/s eta 0:00:011     |███████████████                 | 17.5 MB 22.6 MB/s eta 0:00:01     |████████████████▎               | 19.2 MB 22.6 MB/s eta 0:00:01     |█████████████████               | 19.9 MB 22.6 MB/s eta 0:00:01     |███████████████████▊            | 23.2 MB 22.6 MB/s eta 0:00:01
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760957 sha256=8e05ef348c175cf5df904a1edbd1283a58a5b7cba8695c329b609958e899cbb5
  Stored in directory: /home/jovyan/.cache/pip/wheels/09/14/c5/ea9aee34229caa97c6f6ff78c82c7f2b1a3423c1f37227b6a6
Successfully built snownlp


In [6]:
!pip install jieba

     |████████████████████████████████| 19.2 MB 19.8 MB/s eta 0:00:01
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314478 sha256=850c4c118d38f9ad4f9955295ee676645c61fd549d1d464c8597e7d0231adb2a
  Stored in directory: /home/jovyan/.cache/pip/wheels/ca/38/d8/dfdfe73bec1d12026b30cb7ce8da650f3f0ea2cf155ea018ae
Successfully built jieba


## Load data

In [2]:
from snownlp import SnowNLP
import jieba
import os 
import pandas as pd


pd_corpus = pd.read_csv(csv_path)

print(pd_corpus.head())



FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/work/ten/data/ptt_Salary/processed/Salary.csv'

## Data Cleaning and Tokenizing

In [ ]:
'''
Note: expensive(slow)

'''


import re



def re_filter(x, seed):
    '''
    '''
    
    
#     print(type(x))
    
    for s in re.findall(seed, x):
        if s == "\n":
            x = x.replace(s, " ")
        else:
            x = x.replace(s, "")
    return x

# pd_corpus.title = pd_corpus.title.astype('str')
pd_corpus.content = pd_corpus.content.astype('str')
# pd_corpus.response = pd_corpus.response.astype('str')
 

pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("時間[\w]{3} [\w]{3} [\w]{2} [\w]{2}:[\w]{2}:[\w]{2} [\w]{4}",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("時間[\w]{3} [\w]{3} [\w]{2} [\w]{2} [\w]{4}",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("作者[a-z]*[A-Z]*[0-9]* .*標題",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("[\w]*:",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("發信站: ",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("批踢踢實業坊\(ptt.cc\)|PTT|PTT2|PTT網頁板",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("文章網址",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=(".*網誌.*",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("轉錄至看板|轉錄至某隱形看板",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("※|◆|--|~|→|:|\n|/|\.|=|!",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("  ",)) 

pd_corpus.head()

In [ ]:
'''
Note: very expensive(slow) when use_paddle=True

'''



def to_sentences(x):
    snow = SnowNLP(x)
    S = ""
    for s in snow.sentences:
        S += s
        S += ","
        
    return S

def to_words(x, stop_words):
    words = jieba.cut(x, cut_all=False, use_paddle=False)
#     words = jieba.cut(x, cut_all=False, use_paddle=True)
    W = []
    for w in words:
        if w not in stop_words and w != '\n' and w != ' ':
            W.append(w)
        
    return W

def loadStopWords(path):
    '''
    '''
    
    stopwords = []
    
    with open(path, 'r', encoding='utf-8') as sw:
        for word in sw:
            stopwords.append(word.strip('\n'))
    
    return stopwords

def loadFilterdWord(path):
    '''
    '''
    
    filteredWords = []
    
    with open(path, 'r', encoding='utf-8') as sw:
        for word in sw:
            filteredWords.append(word.strip('\n'))
            
    return filteredWords
            


In [ ]:
stopwords = loadStopWords(path="data/stopwords/stop_words.csv")
print(stopwords)

pd_corpus = pd_corpus[pd_corpus.content!=""]

# Tokenize the contests (documents)
pd_corpus['sentences'] = pd_corpus.content.apply(to_sentences)
pd_corpus['words'] = pd_corpus.sentences.apply(to_words, args=(stopwords,))

print(pd_corpus['sentences'][1])
print(pd_corpus['words'][1])


In [ ]:
pd_corpus.to_csv(w2v_csv_path)

## Tfidf Vectorizer

In [6]:
import pandas as pd
import ast 


pd_corpus = pd.read_csv(w2v_csv_path)

pd_corpus.head()


,Unnamed: 0,content,sentences,words
0,0,今天很高興案子比較少不用加很晚，我和另一個同事A可以先走我才進公司快兩星期而已，A他進來2...,"今天很高興案子比較少不用加很晚,我和另一個同事A可以先走我才進公司快兩星期而已,A他進來2個...","['很', '高興', '案子', '少', '不用', '加', '很', '晚', '一..."
1,1,雖然「理工科薪水高」，聽起來合情合理但我想說的是以平均薪水來說，確實比較高但是我覺得存在嚴...,"雖然「理工科薪水高」,聽起來合情合理但我想說的是以平均薪水來說,確實比較高但是我覺得存在嚴重...","['「', '理工科', '薪水', '高', '」', '聽', '合情合理', '想',..."
2,2,公司因為營運的問題，決定在最近更換營業登記名字。公司上週五宣布這項事情，然後在下班時就寄出...,"公司因為營運的問題,決定在最近更換營業登記名字,公司上週五宣布這項事情,然後在下班時就寄出一...","['公司', '營運', '更換', '營業', '登記', '名字', '公司', '上週..."
3,3,詢問公司相關資訊，請使用 [公司]類別，勿使用 [問題] 類別 確認過後，此兩行得用 ct...,"詢問公司相關資訊,請使用 [公司]類別,勿使用 [問題] 類別 確認過後,此兩行得用 ctr...","['詢問', '公司', '相關', '資訊', '請', '[', '公司', ']', ..."
4,4,"板友們好, 有關這個問題, 已經爬過文, 也google過,不過還是希望能請板友們幫忙看看,...","板友們好, 有關這個問題, 已經爬過文, 也google過,不過還是希望能請板友們幫忙看看,...","['板友們', '好', '爬', '過文', 'google', '希望', '請', '..."


In [7]:
# Select Only unique documents
corpus = pd.DataFrame(pd_corpus.words.sample(frac=1).unique(),
                   columns=['words'])[:20000]

# type converting
corpus.words = corpus.words.apply(lambda x: ast.literal_eval(x))
corpus.words = corpus.words.apply(lambda x: ', '.join(x))

corpus

,words
0,"剛從, 大學畢業, 就讀, 夜間部, 四至, 五年, 會計, 助理, 業務, 助理, 經驗,..."
1,"大哥, 大姐, 您好, 高級, 社區, 保全, 面對, 保全, 操作系統, 社區, 保全, ..."
2,"好, 小弟, 作, 六年, 品保, 公司, 生產, 車用, 零件, 近期, 要換, 工作, ..."
3,"serialhon, 人生如夢, Gossiping, [, 新聞, ], 酒店, 舞廳, ..."
4,"這本, 身, 很, 好笑, 議題, 外勞, 70, 萬, 近, 百萬, 採茶, 工地, 外勞..."
...,...
19995,"小弟, 某間, 公司, 工作, 兩年, 基於, 這間, 公司, 新, 公司, 工號, 最, ..."
19996,"剛從, 公司, 離職, 當初, 離職, 時, 簽的, 自願, 離職, 證明, 打算, 報考,..."
19997,"runa2, あ, ん, た, 飛, ば, し, す, ぎ, Gossiping, [, 新..."
19998,"討論, 串, 看, 這麼久, 原本, 想, 潛水, 好, 難得, 符合, 主題, 就當, 回..."


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer





# compute tf_idf
vectorizer = TfidfVectorizer()
feat_arr = vectorizer.fit_transform(corpus['words'])
tokens = vectorizer.get_feature_names()


In [16]:
feat_arr.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
tokens

['00',
 '000',
 '0000',
 '00000',
 '000000',
 '00000000',
 '0000000100',
 '000000100',
 '0000227',
 '00003000',
 '000050',
 '00008hr',
 '0001',
 '000125',
 '00015',
 '00019',
 '000246',
 '00030',
 '000300150',
 '000301',
 '00032',
 '00037',
 '0004',
 '000400',
 '0005',
 '0007',
 '00070',
 '000b',
 '000month',
 '000nt',
 '000xxxx',
 '00100',
 '00100010',
 '0012',
 '00130',
 '0015',
 '0017',
 '0018',
 '001800',
 '001900001900002000001900',
 '001cm',
 '002',
 '0020',
 '0024',
 '0026202',
 '002644003054',
 '002849',
 '003',
 '0030',
 '00300030',
 '00300045',
 '00303000',
 '004',
 '0040',
 '005',
 '0050',
 '0056',
 '005k',
 '006099',
 '0061512',
 '00713',
 '0078',
 '008',
 '009',
 '00a',
 '00am',
 '00am00pm',
 '00am30am',
 '00pm',
 '00pm00am',
 '00xx',
 '01',
 '010',
 '0100',
 '01000',
 '0101',
 '0103',
 '0109',
 '0110',
 '0111',
 '0112',
 '0113',
 '012',
 '0120',
 '013',
 '01487',
 '016',
 '02',
 '020',
 '0202',
 '0204',
 '0206',
 '0207',
 '0221',
 '0225',
 '022m',
 '025',
 '02k',
 '03',
 

In [20]:
word_idf = pd.DataFrame({'word': vectorizer.get_feature_names(),
                        'tf_idf': feat_arr.toarray().sum(axis=0)})
word_idf

MemoryError: Unable to allocate 14.2 GiB for an array with shape (20000, 95535) and data type float64

In [ ]:
word_idf.sort_values(by='tf_idf', ascending=False)[:10]